# Train MOT17 dataset

Train MOt17 in SageMaker

### Set up

#### 1. Set  up  accounts and role

In [1]:
import sagemaker
import boto3
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

sys.path.append('./src')


sagemaker_session = sagemaker.Session()
account_id =  boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name


#role = sagemaker.get_execution_role()
role="arn:aws:iam::{}:role/service-role/AmazonSageMaker-ExecutionRole-20190118T115449".format(account_id)


#### 2. Setup image and instance type

In [2]:
pytorch_custom_image_name="object-tracking:gpu-1.1.0-201911080239"
instance_type = "ml.p3.2xlarge" 
docker_repo = "{}.dkr.ecr.{}.amazonaws.com/{}".format(account_id, region, pytorch_custom_image_name)

#### 3. Configure train/ test and validation datasets

Set prepare dataset to true if you want to download the 5 GB dataset from Mot 17 and update to s3. Once this is in s3 set the prepare_dataset false to avoid redoing this over and over again

In [3]:
prepare_dataset = False

In [4]:
bucket = sagemaker_session.default_bucket()
raw_bucket="aegovansagemaker"

In [5]:
s3_train="s3://{}/mot17/train/".format(bucket)


s3_val="s3://{}/mot17/val/".format(bucket)

s3_test="s3://{}/mot17/test/".format(bucket)



s3_output_path= "s3://{}/market1501_output/".format(bucket)

## Prepare dataset split

In [6]:
import boto3
import glob
from multiprocessing.dummy import Pool as ThreadPool
import argparse
import datetime 
import os
from datetime import datetime

def uploadfile(localpath, s3path):
        """
Uploads a file to s3
        :param localpath: The local path
        :param s3path: The s3 path in format s3://mybucket/mydir/mysample.txt
        """

        bucket, key = get_bucketname_key(s3path)

        if key.endswith("/"):
            key = "{}{}".format(key, os.path.basename(localpath))
        
        s3 = boto3.client('s3')
        
        s3.upload_file(localpath, bucket, key)

def get_bucketname_key(uripath):
    assert uripath.startswith("s3://")

    path_without_scheme = uripath[5:]
    bucket_end_index = path_without_scheme.find("/")

    bucket_name = path_without_scheme
    key = "/"
    if bucket_end_index > -1:
        bucket_name = path_without_scheme[0:bucket_end_index]
        key = path_without_scheme[bucket_end_index + 1:]

    return bucket_name, key


def download_file(s3path, local_dir):
    bucket, key = get_bucketname_key(s3path)
    
    s3 = boto3.client('s3')
    
    local_file = os.path.join(local_dir, s3path.split("/")[-1])
    

    s3.download_file(bucket, key, local_file)
    
def download_object(s3path):
    bucket, key = get_bucketname_key(s3path)
    
    s3 = boto3.client('s3')    

    s3_response_object = s3.get_object(Bucket=bucket, Key=key)
    object_content = s3_response_object['Body'].read()
    
    return len(object_content)



def list_files(s3path_prefix):
    assert s3path_prefix.startswith("s3://")
    assert s3path_prefix.endswith("/")
    
    bucket, key = get_bucketname_key(s3path_prefix)
    
   
   
    s3 = boto3.resource('s3')
    
    bucket = s3.Bucket(name=bucket)

    return ( (o.bucket_name, o.key) for o in bucket.objects.filter(Prefix=key))





def upload_files(local_dir, s3_prefix, num_threads=20):    
    s3_path = s3_prefix.rstrip("/")
    input_tuples = ( (f, "{}/{}".format( s3_path, f.lstrip(local_dir.lstrip("/")))) for f in glob.glob("{}/**".format(local_dir),  recursive=True) if os.path.isfile(f) )
    print("{} : Uploading {} files".format(datetime.now(), len(list(input_tuples))))
    with ThreadPool(num_threads) as pool:
        pool.starmap(uploadfile, input_tuples)
    print("{} : upload complete".format(datetime.now(), len(list(input_tuples))))
    


def download_files(s3_prefix, local_dir, num_threads=20):    
    input_tuples = ( ("s3://{}/{}".format(s3_bucket,s3_key),  local_dir) for s3_bucket, s3_key in list_files(s3_prefix))
    
    with ThreadPool(num_threads) as pool:
        results = pool.starmap(download_file, input_tuples)
        
        

def download_objects(s3_prefix, num_threads=20):    
    s3_files = ( "s3://{}/{}".format(s3_bucket,s3_key) for s3_bucket, s3_key in list_files(s3_prefix))
    
    with ThreadPool(num_threads) as pool:
        results = pool.map(download_object, s3_files)
        
    return sum(results)/1024
        

def get_directory_size(start_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(start_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            # skip if it is symbolic link
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
    return total_size



In [7]:
from sklearn.model_selection import train_test_split
import shutil
def split_train_val(train_dir_root, dest_train, dest_val, dest_test):
    """
    Expects the root path containing MOT 17 directory structure as is. 
    """
    folders = [ (train_dir_root , d)  for d in os.listdir(train_dir_root) if os.path.isdir(os.path.join(train_dir_root, d))]
    train_list, test_list = train_test_split(    folders , test_size=0.2, random_state=42)
    train_list, val_list = train_test_split(    train_list , test_size=0.1, random_state=42)


    def partition(list_to_partition, dest_dir):
        for root, d  in list_to_partition :
            dest_path = os.path.join(dest_dir, d)
            src_path =  os.path.join(train_dir_root, d)
            print("moving {} to {} ".format(src_path, dest_path))
            shutil.move(src_path, dest_path)
            
    partition(train_list, dest_train)
    partition(val_list, dest_val)
    partition(test_list, dest_test)


    

In [8]:
%%time 

import tempfile

if prepare_dataset:
    # TODO: Download from MOT 17 url
    raw_dataset = "~/Downloads/MOT17/train"
    tmp_dir = tempfile.mkdtemp("mot17data")
    
    #tmp_dir= "/var/folders/sm/0jbn3m7x7gg6sxqyq1h3lmgj6h25fm/T/tmpdl_m8q6wmot17data"
    dest_train = os.path.join(tmp_dir, "train")
    dest_test = os.path.join(tmp_dir, "test")
    dest_val = os.path.join(tmp_dir, "val")
    
    split_train_val(raw_dataset, dest_train, dest_val, dest_test)
    upload_files(dest_train, s3_train)
    upload_files(dest_val, s3_val)
    upload_files(dest_test, s3_test)

else:
    print("Not preparing dataset as prepare_dataset is set to False")

Not preparing dataset as prepare_dataset is set to False
CPU times: user 160 µs, sys: 66 µs, total: 226 µs
Wall time: 187 µs


## Start training

In [9]:
inputs = {
    "train" : s3_train,
    "val" :s3_val
}

In [21]:
hyperparameters = {
    "dataset":"Mot17DetectionFactory",
    "batchsize": "32",
    "epochs" : "1000",
    "learning_rate":.0001,
    "weight_decay":5e-5,
    "momentum":.9,
    "patience": 20,
    "log-level" : "INFO"
}

In [22]:
metric_definitions = [{"Name": "TrainLoss",
                     "Regex": "###score: train_loss### (\d*[.]?\d*)"}
                    ,{"Name": "ValidationLoss",
                     "Regex": "###score: val_loss### (\d*[.]?\d*)"}
                    ,{"Name": "TrainScore",
                     "Regex": "###score: train_score### (\d*[.]?\d*)"}
                   ,{"Name": "ValidationScore",
                     "Regex": "###score: val_score### (\d*[.]?\d*)"}
                    ,{"Name": "trainVariance",
                     "Regex": "###score: train_loss_std### (\d*[.]?\d*)"}
                    ,{"Name": "ValVariance",
                     "Regex": "###score: val_loss_std### (\d*[.]?\d*)"}
                    ]

In [23]:
!git log -1| head -1
!git log -1| tail -1

commit 0a8c60101a232e750f02ee83ec6829fa9fbce51c
    Try  python conda


In [24]:
git_config = {'repo': 'https://github.com/elangovana/object-tracking.git',
              'branch': 'master',
             # 'commit': 'd14df4a3847e74c5672dae42304c0caa5a5c1ae2'
             }

In [25]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
     entry_point='experiment_train.py',
                    source_dir = 'src',
                    dependencies =['src/datasets', 'src/evaluators', 'src/models'],
                    role=role,
                    framework_version ="1.0.0",
                    py_version='py3',
                   # git_config= git_config,
                    image_name= docker_repo,
                    train_instance_count=1,
                    train_instance_type=instance_type,
                    hyperparameters =hyperparameters,
                    output_path=s3_output_path,
                    metric_definitions=metric_definitions,
                    #train_use_spot_instances = True
                    base_job_name ="object-detection")

In [ ]:
estimator.fit(inputs)

2019-11-19 05:29:42 Starting - Starting the training job...
2019-11-19 05:29:44 Starting - Launching requested ML instances...
2019-11-19 05:30:37 Starting - Preparing the instances for training..